<a href="https://colab.research.google.com/github/inventor-2-developer/Amazon_Video_Game_Reviews/blob/main/Amazon_Video_Game_Reviews_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark-Homework").getOrCreate()

## Load Amazon Vine Game Review Data into a Spark DataFrame

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [7]:
from pyspark.sql.functions import avg,sum,max,col,asc,desc,filter,count

import pyspark.sql.functions as func
import pandas as pd

In [8]:

# Write a function that creates a Spark DataFrame that contains the number of times a customer (customer_id) reviewed a product.
# Have the function do the following:
# Count the number of times each customer_id appears
# Rename the count column "customer_counts"
#def customer_counts(df):
#dfGroup=df.groupby('customer_id').agg(count('review_id').alias("customer_counts")).sort(desc('customer_counts')).show()

def customer_counts(df):
  dfGroup=df.groupby('customer_id').agg(count('review_id').alias("customer_counts")).sort(desc('customer_counts')).show()
  return dfGroup



In [9]:
customer_counts(df)

+-----------+---------------+
|customer_id|customer_counts|
+-----------+---------------+
|   52759271|            909|
|   42418272|            856|
|   45838740|            839|
|   24846840|            614|
|   24594001|            460|
|   40955995|            459|
|   48278407|            439|
|   27364030|            361|
|   30678701|            326|
|   50648042|            301|
|   14539589|            288|
|   10541465|            271|
|   38752775|            263|
|   43092063|            257|
|   45261621|            235|
|   14886512|            233|
|   44603910|            233|
|   37687227|            226|
|   10075230|            218|
|   52239651|            214|
+-----------+---------------+
only showing top 20 rows



In [36]:
# Filter the original DataFrame for each star rating, then get the customer counts for each filtered DataFrame 
# by passing the filtered DataFrame into the `customer_counts()` function. 

one_star_ratings=df.filter("star_rating=1")

def customer_counts(one_star_ratings):
  one_star_output=one_star_ratings.groupBy('customer_id').agg(count("star_rating").alias("customer_counts")).sort(desc('customer_counts')).show()
  return one_star_output

In [35]:
customer_counts(one_star_ratings)

+-----------+---------------+
|customer_id|customer_counts|
+-----------+---------------+
|   27364030|            145|
|   47956883|            122|
|   41407731|            105|
|   14000007|             96|
|   14004441|             68|
|   24846840|             65|
|    2061605|             60|
|   10541465|             56|
|   11691516|             56|
|   49047948|             45|
|   44153355|             43|
|   13851096|             41|
|   52340667|             41|
|   13833443|             40|
|   44213519|             38|
|   14539589|             38|
|   38028007|             36|
|   45838740|             33|
|   50648042|             32|
|   48297870|             27|
+-----------+---------------+
only showing top 20 rows



In [33]:
two_star_ratings=df.filter("star_rating=2")

def customer_counts(two_star_ratings):
  two_star_output=two_star_ratings.groupby('customer_id').agg(count('star_rating').alias("customer_counts")).sort(desc('customer_counts')).show()
  return two_star_output


In [34]:
customer_counts(two_star_ratings)

+-----------+---------------+
|customer_id|customer_counts|
+-----------+---------------+
|   45838740|            112|
|   42418272|             59|
|   24476167|             50|
|   52759271|             45|
|   50648042|             45|
|   24846840|             43|
|   30678701|             41|
|   40955995|             38|
|   52239651|             35|
|   45261621|             32|
|   35184137|             31|
|   18069510|             28|
|   52340667|             28|
|   45184473|             25|
|   52546571|             23|
|   12509444|             22|
|   43341147|             22|
|   39576212|             21|
|   41453170|             21|
|   36613238|             19|
+-----------+---------------+
only showing top 20 rows



In [37]:
three_star_ratings=df.filter("star_rating=3")

def customer_counts(three_star_ratings):
  three_star_output=three_star_ratings.groupby('customer_id').agg(count('star_rating').alias("customer_counts")).sort(desc('customer_counts')).show()
  return three_star_output



In [38]:
customer_counts(three_star_ratings)

+-----------+---------------+
|customer_id|customer_counts|
+-----------+---------------+
|   42418272|            271|
|   45838740|            196|
|   40955995|            139|
|   52759271|            104|
|   52239651|             91|
|   30678701|             76|
|   24476167|             70|
|   45261621|             66|
|   48278407|             64|
|   24846840|             62|
|   43092063|             49|
|   41232243|             42|
|   47039038|             41|
|   52125818|             40|
|   16919667|             37|
|   10541465|             36|
|   41545823|             36|
|   41453170|             36|
|   50374480|             34|
|   45654033|             34|
+-----------+---------------+
only showing top 20 rows



In [39]:
four_star_ratings=df.filter("star_rating=4")
def customer_counts(four_star_ratings):
  four_star_output=four_star_ratings.groupby('customer_id').agg(count('star_rating').alias('customer_counts')).sort(desc('customer_counts')).show()
  return(four_star_output)

In [40]:
customer_counts(four_star_ratings)

+-----------+---------------+
|customer_id|customer_counts|
+-----------+---------------+
|   42418272|            451|
|   45838740|            310|
|   52759271|            295|
|   40955995|            203|
|   48278407|            159|
|   24846840|            139|
|   24594001|            111|
|   43092063|            110|
|   35184137|            100|
|   30678701|             99|
|    6010757|             88|
|   38752775|             85|
|   34164491|             78|
|   37687227|             73|
|   14886512|             70|
|   52512615|             68|
|   26131122|             66|
|   38769516|             62|
|   11020935|             56|
|   51397176|             54|
+-----------+---------------+
only showing top 20 rows



In [41]:
five_star_ratings=df.filter("star_rating=5")

def customer_counts(five_star_ratings):
  five_star_output=five_star_ratings.groupby("customer_id").agg(count('star_rating').alias('customer_counts')).sort(desc('customer_counts')).show()
  return five_star_output

In [42]:
customer_counts(five_star_ratings)

+-----------+---------------+
|customer_id|customer_counts|
+-----------+---------------+
|   52759271|            457|
|   24846840|            305|
|   24594001|            304|
|   14539589|            224|
|   44603910|            212|
|   10075230|            211|
|   48278407|            192|
|   45838740|            188|
|   39984534|            186|
|   27364030|            171|
|   38752775|            167|
|   50648042|            146|
|   14886512|            138|
|   26368811|            132|
|   37687227|            127|
|   10541465|            126|
|   15913633|            122|
|   15506517|            116|
|    2335562|            112|
|   28366355|            111|
+-----------+---------------+
only showing top 20 rows

